In [ ]:
pip install altair --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 10.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import altair as alt
import pandas as pd
import numpy as np

df = pd.read_csv('TEPC_2015_Oct_eia.csv')

In [8]:
print(df.head())

             date_time demand (MW) forecast demand (MW)
0  2015-07-01 05:00:00     MISSING              MISSING
1  2015-07-01 06:00:00     MISSING              MISSING
2  2015-07-01 07:00:00     MISSING              MISSING
3  2015-07-01 08:00:00        1605                 1471
4  2015-07-01 09:00:00        1537                 1418


In [5]:
print(df.shape)

(36547, 3)


In [ ]:
df["date_time"] = pd.to_datetime(df["date_time"])
df = df.dropna(subset=["demand (MW)", "forecast demand (MW)"]) # should be filled (with what?)
df['Year'] = df['date_time'].dt.year
df['Month'] = df['date_time'].dt.month
df['Day'] = df['date_time'].dt.day
df['Hour'] = df['date_time'].dt.hour

In [ ]:
sunrise = {
            'January': 8, 'February': 8, 'March': 7, 'April': 7,
            'May': 6, 'June': 6, 'July': 6, 'August': 7, 'September': 7,
            'October': 7, 'November': 8, 'December': 8
        }
sunset = {
            'January': 7, 'February': 7, 'March': 7, 'April': 8,
            'May': 8, 'June': 9, 'July': 9, 'August': 8, 'September': 8,
            'October': 7, 'November': 6, 'December': 6
        }

df['Sunrise'] = df['date_time'].apply(lambda x: sunrise.get((x.month, x.day), np.nan))
df['Sunset'] = df['date_time'].apply(lambda x: sunset.get((x.month, x.day), np.nan))

In [11]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [15]:
# Chart
base = alt.Chart(df).encode(
    x=alt.X("date_time:T", title="Date & Time"),
)

# demand line
actual = base.mark_line(color="steelblue", strokeWidth=2).encode(
    y=alt.Y("demand (MW):Q", title="Electricity Demand (MW)"),
    tooltip=["date_time:T", "demand (MW):Q", "forecast demand (MW):Q"]
)

# forecast line
forecast = base.mark_line(color="orange", strokeDash=[5, 5], strokeWidth=2).encode(
    y="forecast demand (MW):Q"
)

chart = (actual + forecast).properties(
    title="Hourly Electricity Demand: Actual vs Forecast",
    width=1000,
    height=350
).interactive()

chart

alt.LayerChart(...)

In [ ]:
data = df['Month'] # set to month

alt.Chart(data).mark_line().encode(
    x='mean(hour):T',               # avg hourly demand by month
    y="demand (MW):Q",
    color='year:N',
)

(points + text).facet("Month:T")

In [ ]:
# alter this method to create background chart color for daylight
domain = [5, 8, 10, 12, 25]   # <--- change to match sunrise/sunset hours
range_ = ['navy', 'royalblue', 'skyblue', 'lightyellow', 'skyblue', 'royalblue', 'navy']

alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color=alt.Color('Acceleration').scale(domain=domain, range=range_)     # <--- use this
)

background = alt.Chart(df).mark_rect(opacity=0.1).encode(
    x= [0, 24], 
    y= [0, 5000],
    color=alt.Color('Hour:Q', scale=alt.Scale(domain=domain, range=range_))   # <--- use this
)

Base chart structure achieved. 

TODO:
* facet to show hourly demand over months
* facet to show hourly demand over year
* add error line
